# 이진 분류 예측
Data Science London + Scikit-learn

- Data Science London은 Scikit-learn에서 모임 주최중
- Scikit-learn을 활용한 분류 능력에 대한 사례를 연습하기 위함
    - Scikit-learn(sklearn)은 NumPy, SciPy 및 Cython의 도움을 받아 Python으로 작성된 확립된 오픈 소스 기계 학습 라이브러리
    - Scikit-learn은 매우 사용자 친화적이고 일관된 API를 갖추고 있으며 광범위한 문서를 제공
    -  엄격한 코딩 표준과 높은 테스트 범위로 인해 구현 품질이 높음
    - sklearn 뒤에는 매우 활동적인 커뮤니티가 있어 꾸준한 라이브러리 개선




# Data Science London + Scikit-learn는 어떤 대회인가?
1. 데이터 세트
레이블이 지정된 훈련 데이터 세트와 레이블이 없는 테스트 데이터 세트로 구성
훈련 데이터를 사용해 모델을 학습 시키고, 테스트 데이터에 대한 예측 생성해야 함

2. 목표
주어진 데이터를 바탕으로 최적의 ML 모델을 구축하여 테스트 데이터에 대한 예측을 하는 것

3. 도구
Scikit-learn 같은 Python 라이브러리가 주로 사용됩니다. 이 라이브러리는 분류, 회귀, 군집화 등 다양한 머신러닝 기법을 지원

4. 참가자
이 대회는 데이터 과학과 머신러닝 분야의 초보자들에게 인기가 있으며, 실제 프로젝트에 적용할 수 있는 경험을 쌓을 수 있는 좋은 기회를 제공

5. 평가 기준
평가 방법은 대회마다 다를 수 있으나, 일반적으로 정확도, F1 점수, ROC-AUC 점수 등의 메트릭을 사용하여 모델의 성능을 평가

In [9]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV  
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import cross_val_score  
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [11]:
# 훈련 데이터(x_train), 훈련 레이블(y_train), 테스트 데이터(x_test)를 CSV 파일에서 불러오기
# 데이터는 NumPy 배열로 변환되고, y_train은 1차원 배열로 변형

x_train = pd.read_csv('C:/Users/jyjj0/OneDrive/바탕 화면/Algorithm Study/WEEK13/train.csv')
y_train = pd.read_csv('C:/Users/jyjj0/OneDrive/바탕 화면/Algorithm Study/WEEK13/trainLabels.csv')
x_test = pd.read_csv('C:/Users/jyjj0/OneDrive/바탕 화면/Algorithm Study/WEEK13/test.csv')

In [12]:
# 훈련 데이터와 테스트 데이터를 결합하여 x_all을 생성
# 이는 Gaussian Mixture Model(GMM)을 적용하기 위한 전처리 단계

x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
x_test = np.asarray(x_test)
y_train = y_train.ravel()

In [13]:
#다양한 구성 요소 수(n_components)와 공분산 유형(cv_types)에 대해 GMM을 적용
#Akaike Information Criterion(AIC)를 사용하여 최적의 모델을 선택
#선택된 최적의 GMM 모델로 x_train과 x_test 데이터를 변환
#데이터 크기 출력:변환된 데이터의 차원(행과 열의 수)을 출력


print('training_x Shape:', x_train.shape, ',training_y Shape:', y_train.shape, ',testing_x Shape:', x_test.shape)

training_x Shape: (999, 40) ,training_y Shape: (999,) ,testing_x Shape: (8999, 40)


In [14]:
# K-Nearest Neighbors(KNN) 분류기와 Random Forest 분류기를 초기화
# GridSearchCV를 사용하여 각 모델에 대한 최적의 매개변수를 찾음 -> 여기서는 매개변수 그리드가 비어 있으므로 기본 매개변수 사용
#훈련 및 테스트 데이터 결합:np.r_을 사용하여 훈련 데이터와 테스트 데이터를 결합
#결합된 데이터 x_all의 크기를 출력

x_all = np.r_[x_train, x_test]
print('x_all shape:', x_all.shape)

x_all shape: (9998, 40)


Using Gaussian Mixture Model to transform features 

Gaussian Mixture Model(GMM) 적용


GMM 구성 요소(n_components) 및 공분산 유형(cv_types)에 대해 모델을 학습시킴

각 모델의 AIC 값을 계산하여 최적의 GMM 모델을 선택

In [15]:
lowest_bic = np.infty
bic = []
n_components_range = range(1, 7)
cv_types = ['spherical', 'tied', 'diag', 'full']

for cv_type in cv_types:
    for n_components in n_components_range:
        gmm = GaussianMixture(n_components=n_components, covariance_type=cv_type)
        gmm.fit(x_all)
        bic.append(gmm.aic(x_all))
        if bic[-1] < lowest_bic:
            lowest_bic = bic[-1]
            best_gmm = gmm

best_gmm.fit(x_all)
x_train = best_gmm.predict_proba(x_train)
x_test = best_gmm.predict_proba(x_test)

In [16]:
# 모델 초기화
knn = KNeighborsClassifier()
rf = RandomForestClassifier()

In [17]:
# Grid Search for best parameters for KNN
grid_search_knn = GridSearchCV(knn, param_grid={}, cv=10, scoring='accuracy').fit(x_train, y_train)
print('best estimator KNN:', grid_search_knn.best_estimator_, 'Best Score', grid_search_knn.best_estimator_.score(x_train, y_train))
knn_best = grid_search_knn.best_estimator_

best estimator KNN: KNeighborsClassifier() Best Score 0.995995995995996


최적화된 KNN과 랜덤 포레스트 모델을 훈련 데이터에 적합시킴

테스트 데이터에 대해 예측을 수행하고 처음 10개의 예측 결과를 출력

In [18]:
# Grid Search for best parameters for Random Forest
grid_search_rf = GridSearchCV(rf, param_grid={}, verbose=3, scoring='accuracy', cv=10).fit(x_train, y_train)
print('best estimator RandomForest:', grid_search_rf.best_estimator_, 'Best Score', grid_search_rf.best_estimator_.score(x_train, y_train))
rf_best = grid_search_rf.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV 1/10] END .................................., score=1.000 total time=   0.0s
[CV 2/10] END .................................., score=1.000 total time=   0.0s
[CV 3/10] END .................................., score=1.000 total time=   0.0s
[CV 4/10] END .................................., score=0.990 total time=   0.0s
[CV 5/10] END .................................., score=1.000 total time=   0.0s
[CV 6/10] END .................................., score=1.000 total time=   0.0s
[CV 7/10] END .................................., score=0.990 total time=   0.0s
[CV 8/10] END .................................., score=1.000 total time=   0.0s
[CV 9/10] END .................................., score=0.980 total time=   0.0s
[CV 10/10] END ................................., score=1.000 total time=   0.0s
best estimator RandomForest: RandomForestClassifier() Best Score 0.997997997997998


In [22]:
# 최적화된 KNN 모델과 rf모델을 각각 훈련 데이터에 적합시키고, 테스트 데이터에 대한 예측을 수행함. 예측 결과의 처음 10개를 출력
knn_best.fit(x_train, y_train)
print(knn_best.predict(x_test)[:10])

rf_best.fit(x_train, y_train)
print(rf_best.predict(x_test)[:10])

[0 1 0 0 0 0 1 0 0 1]
[0 1 0 0 0 0 1 0 0 1]


In [20]:
# 교차 검증을 통한 모델 평가
print('Score for KNN:', cross_val_score(knn_best, x_train, y_train, cv=10, scoring='accuracy').mean())
print('Score for Random Forest:', cross_val_score(rf_best, x_train, y_train, cv=10, scoring='accuracy').max())

Score for KNN: 0.9960000000000001
Score for Random Forest: 1.0


교차 검증을 사용하여 KNN과 랜덤 포레스트 모델의 정확도를 평가

여기서는 평균 정확도와 최대 정확도를 출력

In [21]:
# Saving predictions to CSV
rf_best_pred = pd.DataFrame(rf_best.predict(x_test))
rf_best_pred.index += 1
rf_best_pred.to_csv('Submission.csv')

 GMM을 사용하여 데이터의 특성을 변환하고, KNN과 랜덤 포레스트 분류기를 통해 분류 문제를 해결하는 전략을 보여줌
 
 코드의 효율성과 정확도는 데이터의 특성과 매개변수 설정에 크게 의존함